In [ ]:
import os
from agents import Runner, agent, trace, set_trace_processors, Agent, ModelSettings
from weave.integrations.openai_agents.openai_agents import WeaveTracingProcessor
import asyncio
from typing import Literal, List, Annotated
import weave
from dotenv import load_dotenv
from pydantic import BaseModel, Field, conlist
from IPython.display import display, Markdown, HTML, Image
load_dotenv()
# Import Necessary Libraries
from src.Tools.Agentic_Calculator_Tool import Agentic_Calculator_Tool
from src.Tools.PerplexitySECSonarPro_Tool import PerplexitySECSonarPro_Tool
from src.Tools.Search_Tool import Search_Tool
from src.Tools.OpenAIDeepResearch_Tool import OpenAIDeepResearch_Tool
# Import Weave and Biases
weave.init("Operation Model")
set_trace_processors([WeaveTracingProcessor()])

## Analysis Q&A Format

In [ ]:
ANALYSIS_QA_PROMPT = '''
System= You are a Generative AI analyst that provides scenarios output.

# Scenarios
1. High Scenario assumes that Generative AI will severely impact the process.
2. Medium Scenario assumes that Generative AI will have moderate process on the scenarios.
3. Low Scenario assumes that Generative AI will have minimal impact on the scenarios.

# Output Method
1. Provide a detailed 5-year cumulative vector percentage change for each of the scenarios.

# Instruction
1. Thoroughly understand the process and information provided by the user.
2. Use the <PerplexitySECSonarPro_Tool> to understand the detailed processes within a bank for the user original input.
3. Use the <PerplexitySECSonarPro_Tool> to understand how the detailed process will be impacted by Generative AI in banks.
4. Use the <Search_Tool> to search the internet for any additional information required.
5. Use the <Agentic_Calculator_Tool> to evaluate the appropriateness of Generative AI for a specific business activity.
6. Use the <OpenAIDeepResearch_Tool> to understand the impact of Generative AI on the business process.
6. Provide a detailed 5-year cumulative vector percentage change for each of the scenarios.

# Tool Description
1. <PerplexitySECSonarPro_Tool>: Provides research capabilities for Security Exchange Commission (SEC) fillings.
2. <Agentic_Calculator_Tool>: Provides a tool for evaluating the appropriateness of Generative AI for a specific business activity.
3. <Search_Tool>: Provides a tool for searching for information on a given topic.
4. <OpenAIDeepResearch_Tool>: OpenAIDeepResearch_Tool provides research capabilities for modeling how Generative AI will impact the business process in the next 5-years.
'''

class Agentic_Calculator_Tool_Output(BaseModel):
    high_scenario: conlist(float, min_length=5, max_length=5)
    "High-scenario cumulative vector percentage change"
    medium_scenario: conlist(float, min_length=5, max_length=5)
    "Medium-scenario cumulative vector percentage change"
    low_scenario: conlist(float, min_length=5, max_length=5)
    "Low-scenario cumulative vector percentage change"
    high_scenario_reasoning: str
    "Provide a detailed reasoning chain of thought for the high-scenario cumulative vector percentage change"
    medium_scenario_reasoning: str
    "Provide a detailed reasoning chain of thought for the medium-scenario cumulative vector percentage change"
    low_scenario_reasoning: str
    "Provide a detailed reasoning chain of thought for the low-scenario cumulative vector percentage change"
    hallucination_score: Literal["Low", "Medium", "High"]
    "Provide a score of the response quality. Low = Low hallucination risk, Medium = Medium hallucination risk, High = High hallucination risk"

q_a_agent = Agent(
    name="q_a_agent",
    instructions=ANALYSIS_QA_PROMPT,
    output_type=Agentic_Calculator_Tool_Output,
    model=os.getenv("LLM_MODEL"),
    model_settings=ModelSettings(reasoning={"effort": "high"}),
    tools=[
            PerplexitySECSonarPro_Tool.as_tool(
                tool_name="PerplexitySECSonarPro_Tool",
                tool_description="PerplexitySECSonarPro_Tool provides research capabilities for Security Exchange Commission (SEC) fillings."
    ),
           Agentic_Calculator_Tool.as_tool(
               tool_name="Agentic_Calculator_Tool",
               tool_description="Agentic_Calculator_Tool provides a tool for evaluating the appropriateness of Generative AI for a specific business activity."
           ),
           Search_Tool.as_tool(
               tool_name="Search_Tool",
               tool_description="Search_Tool provides a tool for searching for information on a given topic."
           ),
        OpenAIDeepResearch_Tool.as_tool(
            tool_name="OpenAIDeepResearch_Tool",
            tool_description="OpenAIDeepResearch_Tool provides research capabilities for modeling how Generative AI will impact the business process in the next 5-years."
        )
    ],
)

response = await Runner.run(q_a_agent, "Mortgage Services in Load Admin")

In [ ]:
response.final_output